In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st 
#Graphing libraries
#import for seaborn graphs
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.io as pio
pio.templates.default = "plotly_dark"


In [5]:
#Retriveing the data from Output folder and combining into the dataframe and assigning variables
#using loop to retrives 5 csv files.
years = [x for x in range(2015,2020)]
df={}
for year in years:
    url = f"../Output/{year}.csv"
    df[year] = pd.read_csv(url)
#Retreivng the concated csv file    
url_concat = f"../Output/concated_df.csv"
df_concat = pd.read_csv(url_concat)
#Unpacking the dataframe
df_2015,df_2016,df_2017,df_2018,df_2019 = df[2015],df[2016],df[2017],df[2018],df[2019]

FileNotFoundError: [Errno 2] No such file or directory: '../Output/concated_df.csv'

In [ ]:
df[2016].tail(10)

# Horizontal Bars

In [ ]:
#Creating empty dictionioraing to hold top 10 and botton countries
top_10 = {}
bottom_10 = {}
combined_10 = {}

#using for loop to get the df for 5 years
for year in years:
        top_10[year] =df[year].set_index('Country')
        #Using loc for columns having significant factor for happiness score
        #Top 10 Countries
        top_10[year] = df[year].loc[:,['Country','Economy_(GDP_per_Capita)','Family', 'Health_(Life_Expectancy)',
       'Freedom', 'Trust_(Government_Corruption)', 'Generosity',
       'Dystopia_Residual']].head(10)
         #Using loc for columns having significant factor for happiness score
        #bottom 10 Countries
        bottom_10[year] = df[year].set_index('Country')
        bottom_10[year] = df[year].loc[:,['Country','Economy_(GDP_per_Capita)','Family', 'Health_(Life_Expectancy)',
       'Freedom', 'Trust_(Government_Corruption)', 'Generosity',
       'Dystopia_Residual']].tail(10)
        #combing to to and bottom to get in the same graphs
        combined_10[year] = pd.concat([bottom_10[year][::-1],top_10[year][::-1]]).set_index('Country',drop=True)
        fig =px.bar(combined_10[year], color_discrete_sequence = px.colors.colorbrewer.Paired,orientation='h',title= f'{year} Top-Bottom Analysis',barmode='stack', labels=dict(Country="Countries", value="Happiness Score", variable="Contributing Factors"))
        fig.show()


# Correlational Heatmaps

In [ ]:
#deciding the size of graphs
plt.figure(figsize=(10,6))
heatmap_corr={}
# Using for loops to map co-relations grpahs for 5 years
for year in years:
    heatmap_corr[year] = df[year]
    fig = plt.figure(figsize=(10,6))
    print(f"Cor-relational heatmap - {year}")
    #Seaborn graphing using heatmap
    sns.heatmap(heatmap_corr[year].corr(), annot=True)
    plt.show()
    
  

### Spider Graphs

In [ ]:

categories = ['Economy_(GDP_per_Capita)', 'Family', 'Health_(Life_Expectancy)',
       'Freedom', 'Trust_(Government_Corruption)', 'Generosity']
#plotlly function for go graphs
fig = go.Figure()
#empty dictionary to hold australian column values for the seven aforementioned categories
aus_data ={}
top_data = {}
country_name = {}
for year in years:
    #filtering out australian rows every years
    aus_data[year] = df[year].loc[df[year]['Country']=='Australia'].values
    #filtering out the top country values for evry years
    top_data[year]  = df[year].iloc[0,:].values
    #storing country name for all the years
    country_name[year]  = top_data[year][0]
  
    
fig.add_trace(go.Scatterpolar(
    #plotting top country data for 2019
          r=top_data[2019][3:-1],
    #adding afforentioned categories
          theta=categories,
          fill='toself',
          name=f'{country_name[year]}'
    ))
fig.add_trace(go.Scatterpolar(
    #plotting autrlaia data for 2019
          r=aus_data[2019][0][3:-1],
          theta=categories,
          fill='toself',
          name='Australia'
    ))

fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[0, 2]
        )),
      title=f"{country_name[2019]} vs. Australia - 2019",
      showlegend=True
    )

fig.show()



In [ ]:
fig = go.Figure()
   
fig.add_trace(go.Scatterpolar(
    #plotting top country data for 2015
          r=top_data[2015][3:-1],
          theta=categories,
          fill='toself',
          name=f'{country_name[2015]}'
    ))
fig.add_trace(go.Scatterpolar(
    #plotting autrlaia data for 2015
          r=aus_data[2015][0][3:-1],
          theta=categories,
          fill='toself',
          name='Australia'
    ))

fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          
          range=[0, 2]
        )),
       title=f"{country_name[2015]} vs. Australia - 2015",
      showlegend=True
    )

fig.show()



### True Factor Graph

In [ ]:
#fill True Factor column by finding the max value from the database
df_concat['True_Factor']=df_concat.loc[:,['Economy_(GDP_per_Capita)','Family','Health_(Life_Expectancy)','Freedom','Trust_(Government_Corruption)','Generosity']].idxmax(axis=1)

In [ ]:
#plotly chrolopleth graph
fig = px.choropleth(df_concat,  locations='Country',locationmode='country names', color='True_Factor',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="world",
                    animation_frame = 'year',
                           labels={'True_Factor':'True Factor'}
                          )

fig.show()

## Happiness Ranking of Top 10 Countries from 2015 to 2019

#### Observations

* Finland is the Country showed steady progress and is consistently Rank 1 in 2018 and 2019

* Denmark keeps in first 3 positions from 2015 to 2019. In 2016, they were Rank 1
 
* Switzerland was the Rank 1 in 2015 but went down is Rank 6 in 2019

* The land of Midnight sun, Norway was Rank 1 in 2017

* Australia was in top 10 Countries but not in there, last year. Austria comes instead.


In [ ]:
# Creating a dataframe with only Records of top 10 ranks in each year
df = df_concat.loc[df_concat["Happiness_Rank"] <= 10].sort_values(by=['year'])

# Getting the countries list
countries = df_concat.loc[df_concat["Happiness_Rank"] <= 10]['Country'].unique()

# Creating a figure
fig = go.Figure()

# Looping through each country to plot its ranking from 2015 to 2019
for country in countries: 
    
    # Setting x values as Year and y values as Happiness_Rank from the data frame
    x = df.loc[df['Country']==country]['year']
    y = df.loc[df['Country']==country]['Happiness_Rank']
    
    # Ploting the scatter + lines
    fig.add_trace(go.Scatter(x=x, y=y,
                        mode='markers+lines',
                        line=dict(width=5, dash='solid'),
                        marker=dict(size=10,symbol='square',),
                        name=country),
                 )
    
# Changing the figure style 
fig.update_xaxes(nticks=5)
fig.update_yaxes(tick0=1, dtick=1, autorange = "reversed")
fig.update_layout(
    title="Happiness Ranking of Top 10 Countries from 2015 to 2019",
    xaxis_title="Year",
    yaxis_title="Happiness Rank",
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#ffffff"
    )
)
# import plotly.io as pio
# pio.write_image(fig, '../Images/ranking.png')


fig.show()

### Rye Code

In [79]:
# Creating a dataframe with only Records of top 10 ranks in each year
top_10_2015 = df_2015.loc[df_2015["Happiness_Rank"] <= 10, :]
top_10_2016 = df_2016.loc[df_2016["Happiness_Rank"] <= 10, :]
top_10_2017 = df_2017.loc[df_2017["Happiness_Rank"] <= 10, :]
top_10_2018 = df_2018.loc[df_2018["Happiness_Rank"] <= 10, :]
top_10_2019 = df_2019.loc[df_2018["Happiness_Rank"] <= 10, :]


In [80]:
# Create a List and store the Australia scores and Top 10 minimum scores of each year
year_count = [2015, 2016, 2017, 2018, 2019]
au_scores = []
min_happ_score = []
au_scores.append(top_10_2015.iloc[9, 2])
au_scores.append(top_10_2016.iloc[8, 2])
au_scores.append(top_10_2017.iloc[9, 2])
au_scores.append(top_10_2018.iloc[9, 2])
au_scores.append(df_2019.iloc[10, 2])
min_happ_score.append(top_10_2015["Happiness_Score"].min())
min_happ_score.append(top_10_2016["Happiness_Score"].min())
min_happ_score.append(top_10_2017["Happiness_Score"].min())
min_happ_score.append(top_10_2018["Happiness_Score"].min())
min_happ_score.append(top_10_2019["Happiness_Score"].min())

# Create a DataFrame for the data
au_pd = pd.DataFrame({"Year": year_count,
                     "AU_Happiness_Score": au_scores,
                     "Top_10_score": min_happ_score})

In [81]:
au_pd

,Year,AU_Happiness_Score,Top_10_score
0,2015,7.284,7.284
1,2016,7.313,7.291
2,2017,7.284,7.284
3,2018,7.272,7.272
4,2019,7.228,7.246


In [85]:
# Australia linear equation & prediction score in 2020 using plotly
x_axis = au_pd["Year"]
y_axis = au_pd["AU_Happiness_Score"]
au_2020 = 2020
slope, intercept, r_value, p_value, std_err = st.linregress(x_axis, y_axis)
au_predicted_score = round(slope * au_2020 + intercept,2)
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

fig = px.scatter(au_pd, x = "Year", y = "AU_Happiness_Score", opacity=1, 
    trendline = "ols", trendline_color_override = "red")
fig.update_xaxes(nticks=5)
fig.update_layout(height=400, title_text = "Australia's Happiness Score in years",
                annotations = [dict(x = 2016, y = 7.26, text = line_eq, font_size = 20, font_color = "red",
                              showarrow = False), 
                              dict(x = 2016, y = 7.25, text = f"r value is: {round(r_value, 2)}",
                              font_size = 20, font_color = "black", showarrow = False), 
                              dict(x = 2016, y = 7.24, text = f"r^2 value is: {round(r_value**2, 2)}",
                              font_size = 20, font_color = "green", showarrow = False),  
                              dict(x = 2016, y = 7.23, text = f"Predicted score in 2020: {au_predicted_score}", 
                              font_size = 20, font_color = "blue", showarrow = False)
                              ])
fig.show()



In [84]:

# Top 10 minimum score linear equation & prediction score in 2020 using plotly
x_axis = au_pd["Year"]
y_axis = au_pd["Top_10_score"]
top_10_2020 = 2020
slope, intercept, r_value, p_value, std_err = st.linregress(x_axis, y_axis)
top_10_predicted_score = round(slope * top_10_2020 + intercept,2)
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

fig = px.scatter(au_pd, x = "Year", y = "Top_10_score", opacity=1, 
    trendline = "ols", trendline_color_override = "red")
fig.update_xaxes(nticks=5)
fig.update_layout(height=400, title_text = "Top 10th Country Happiness Score in years",
                annotations = [dict(x = 2016, y = 7.265, text = line_eq, font_size = 20, font_color = "red",
                              showarrow = False), 
                              dict(x = 2016, y = 7.26, text = f"r value is: {round(r_value, 2)}",
                              font_size = 20, font_color = "black", showarrow = False), 
                              dict(x = 2016, y = 7.255, text = f"r^2 value is: {round(r_value**2, 2)}",
                              font_size = 20, font_color = "green", showarrow = False),   
                              dict(x = 2016, y = 7.25,
                              text = f"Predicted top 10th score in 2020: {top_10_predicted_score}",
                              font_size = 20, font_color = "blue", showarrow = False)])

fig.show()
